In [ ]:
from transformers import pipeline

# 加载情感分析 pipeline
classifier = pipeline("sentiment-analysis", model="bert-base-uncased")

# 推理
result = classifier("This movie was awesome!")
print(result)  # [{'label': 'POSITIVE', 'score': 0.999...}]


In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("fill-mask", model="google-bert/bert-base-chinese")

result = pipe("我是一个[MASK]")
print(result)

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch

# 加载模型和 tokenizer
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-chinese")
model = AutoModelForMaskedLM.from_pretrained("google-bert/bert-base-chinese")

# 设置模型为评估模式
model.eval()

# 输入文本：注意 [MASK] 是英文方括号加大写字母
text = "我喜欢吃[MASK]。"

# 编码
inputs = tokenizer(text, return_tensors="pt")

# 获取模型输出
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

# 找到 [MASK] 位置
mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]

# 获取该位置预测结果，取概率最高的前 5 个词
top_k = 5
mask_logits = logits[0, mask_token_index, :]
top_tokens = torch.topk(mask_logits, top_k, dim=1).indices[0].tolist()

print(f"原句：{text}")
print("预测：")
for token_id in top_tokens:
    token = tokenizer.decode([token_id])
    print(f" -> 我喜欢吃{token}。")
